# Evolver Loop 11 Analysis

## Situation Summary
- Current best: 70.630478 (verified on LB)
- Target: 68.919154
- Gap: 1.711 points (2.42%)
- Submissions used: 6/100 (84 remaining)

## Key Insights from 12 Experiments
1. All public sources exhausted - ceiling is 70.630478
2. Simple lattice approaches (hexagonal, square, interlocking) are WORSE
3. Current solutions use sophisticated multi-angle optimization (not 0/180)
4. Tree deletion and add-delete approaches don't improve
5. The C++ optimizer (sa_v1_parallel) has been run but not for long enough

In [1]:
import pandas as pd
import numpy as np

# Load current best
df = pd.read_csv('/home/code/exploration/datasets/saspav_best.csv')

def strip(a):
    return np.array([float(str(v).replace('s','')) for v in a], np.float64)

def score_group(xs, ys, degs):
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    tx=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    ty=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    
    n = len(xs)
    mnx=1e300; mny=1e300; mxx=-1e300; mxy=-1e300
    for i in range(n):
        r = degs[i] * np.pi / 180.0
        c = np.cos(r); s = np.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(len(tx)):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx-mnx, mxy-mny)
    return side*side / n

df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)

scores = []
for n in range(1, 201):
    g = df[df['N'] == n]
    xs = strip(g['x'].values)
    ys = strip(g['y'].values)
    ds = strip(g['deg'].values)
    sc = score_group(xs, ys, ds)
    scores.append((n, sc))

print('Score distribution by N range:')
for start, end in [(1, 20), (21, 50), (51, 100), (101, 150), (151, 200)]:
    range_scores = [s for n, s in scores if start <= n <= end]
    print(f'  N={start:3d}-{end:3d}: sum={sum(range_scores):.4f}, avg={np.mean(range_scores):.6f}')

print(f'\nTotal: {sum(s for _, s in scores):.6f}')
print(f'Target: 68.919154')
print(f'Gap: {sum(s for _, s in scores) - 68.919154:.6f}')

Score distribution by N range:
  N=  1- 20: sum=8.0532, avg=0.402660
  N= 21- 50: sum=10.9809, avg=0.366031
  N= 51-100: sum=17.6170, avg=0.352340
  N=101-150: sum=17.1364, avg=0.342728
  N=151-200: sum=16.8430, avg=0.336859

Total: 70.630478
Target: 68.919154
Gap: 1.711324


In [2]:
# Key insight: The gap is 1.711 points (2.42%)
# This is a SIGNIFICANT gap that requires either:
# 1. MUCH longer optimization runs (hours, not minutes)
# 2. A fundamentally different approach

# The jonathanchan kernel parameters:
# - N <= 20: r = max(6, nr), it = int(si * 1.5) = 30000 iterations, 80 restarts
# - N <= 50: r = max(5, nr), it = int(si * 1.3) = 26000 iterations, 80 restarts  
# - N > 150: r = max(4, nr), it = int(si * 0.8) = 16000 iterations, 80 restarts

# Our runs: n=30000, r=5 (4 generations)
# jonathanchan: n=20000-30000, r=80 (10+ generations)

print('Parameter comparison:')
print('='*60)
print('Our runs:')
print('  iterations: 30000')
print('  restarts: 5')
print('  generations: 4')
print()
print('jonathanchan kernel:')
print('  iterations: 20000-30000 (per-N scaling)')
print('  restarts: 80')
print('  generations: 10+')
print()
print('KEY DIFFERENCE: We need 16x more restarts!')

Parameter comparison:
Our runs:
  iterations: 30000
  restarts: 5
  generations: 4

jonathanchan kernel:
  iterations: 20000-30000 (per-N scaling)
  restarts: 80
  generations: 10+

KEY DIFFERENCE: We need 16x more restarts!


In [3]:
# STRATEGY RECOMMENDATION:
#
# The evaluator is correct - we need to run the optimizer for MUCH longer
# with more restarts. The key parameters are:
#
# 1. Restarts: 80 (not 5)
# 2. Iterations: 20000-30000 (per-N scaling)
# 3. Generations: 10+ (not 4)
#
# However, there's a risk: if the solution is truly at a local optimum,
# more restarts won't help. We need to try DIFFERENT initial configurations.
#
# ALTERNATIVE APPROACH: Perturb the current best and re-optimize
# The jonathanchan kernel does this with the perturb() function:
# - Randomly perturb positions by 0.1-0.15
# - Randomly perturb angles by 10-20 degrees
# - Re-optimize from the perturbed state

print('RECOMMENDED NEXT EXPERIMENT:')
print('='*60)
print('1. Run sa_v1_parallel with MUCH higher restarts:')
print('   ./sa_v1_parallel -i current_best.csv -o output.csv -n 30000 -r 80')
print()
print('2. Let it run for 10+ generations (may take 2-4 hours)')
print()
print('3. If no improvement, try perturbing the current best:')
print('   - Add random noise to positions (0.1-0.15)')
print('   - Add random noise to angles (10-20 degrees)')
print('   - Re-optimize from perturbed state')
print()
print('4. Submit the result to get LB feedback')

RECOMMENDED NEXT EXPERIMENT:
1. Run sa_v1_parallel with MUCH higher restarts:
   ./sa_v1_parallel -i current_best.csv -o output.csv -n 30000 -r 80

2. Let it run for 10+ generations (may take 2-4 hours)

3. If no improvement, try perturbing the current best:
   - Add random noise to positions (0.1-0.15)
   - Add random noise to angles (10-20 degrees)
   - Re-optimize from perturbed state

4. Submit the result to get LB feedback
